In [3]:
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import re
import ast
import gower

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score, pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from scipy.stats import chi2_contingency
from scipy.spatial import distance
from joblib import dump, load

# modele
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, HDBSCAN
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier, NearestNeighbors
from sklearn.inspection import permutation_importance
from scipy.spatial.distance import euclidean
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
#from ctgan import CTGAN

In [41]:
#oversampling data
original_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\original_data.csv")    
original_data = original_data.drop(columns=["Unnamed: 0"])
smote_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\smote_data.csv")
GAN_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\GAN_data.csv")
borderline_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\borderline_data.csv")
smote2_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\smote3_data.csv")
GAN2_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\GAN3_data.csv")    
borderline2_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\borderline3_data.csv")

# test data
X_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\test\\X_test.csv")
y_test = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\test\\y_test.csv")

# load scalar and encoder map 
with open("D:\\ml\\undersampling_data\\models\\scaler_ip_no_td.pkl", "rb") as f:
    scalar_ip_no_td = pickle.load(f)
with open("D:\\ml\\undersampling_data\\models\\map_user.pkl", "rb") as f:
    reverse_user_map = pickle.load(f)

#Before undersampling
print(original_data["target"].value_counts())
count1=original_data["target"].value_counts().sum()
print(count1)
count2=abs((original_data['target']==0).sum() - (original_data['target']==1).sum())
print(f"Before undersampling: {count2}")
print(f"After number of samples: {count1+count2}")

data_oririnal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\SSH.csv")

target
0    151
1     46
Name: count, dtype: int64
197
Before undersampling: 105
After number of samples: 302


C:\Users\mateu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.0 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [56]:
#mixed data


#data with one oversampling method and original data e.g.(smote+original)
smote_data = pd.concat([smote_data, smote2_data], axis=0, ignore_index=True)
smote_data = smote_data.reset_index(drop=True)
borderline_data = pd.concat([borderline_data, borderline2_data], axis=0, ignore_index=True)
borderline_data = borderline_data.reset_index(drop=True)
GAN_data = pd.concat([GAN_data, GAN2_data], axis=0, ignore_index=True)
GAN_data = GAN_data.reset_index(drop=True)

mix_data = pd.concat([GAN_data, smote_data, borderline_data], axis=0, ignore_index=True)         
mix_data = mix_data.reset_index(drop=True)

#convert data types to float64
int_cols = mix_data.select_dtypes(include=["int"]).columns
mix_data[int_cols] = mix_data[int_cols].astype("float64")
int_cols = smote_data.select_dtypes(include=["int"]).columns
smote_data[int_cols] = smote_data[int_cols].astype("float64")
int_cols = borderline_data.select_dtypes(include=["int"]).columns
borderline_data[int_cols] = borderline_data[int_cols].astype("float64")
int_cols = GAN_data.select_dtypes(include=["int"]).columns
GAN_data[int_cols] = GAN_data[int_cols].astype("float64")

sum_all_data = pd.concat([smote_data, GAN_data, borderline_data, original_data], axis=0, ignore_index=True)
sum_all_data = sum_all_data.drop_duplicates()


#Split data
X_mix, y_mix = mix_data.drop(columns=["target", "source"]), mix_data["target"]
X_smote, y_smote = smote_data.drop(columns=["target", "source"]), smote_data["target"]
X_GAN, y_GAN = GAN_data.drop(columns=["target", "source"]), GAN_data["target"]
X_borderline, y_borderline = borderline_data.drop(columns=["target", "source"]), borderline_data["target"]

#dodac standrazycje 

#Dictionary
data = {}
data["mix"] = (X_mix, y_mix)
data["smote"] = (X_smote, y_smote)
data["GAN"] = (X_GAN, y_GAN)
data["borderline"] = (X_borderline, y_borderline)

compare = {}
compare["mix"] = mix_data
compare["smote"] = smote_data
compare["GAN"] = GAN_data
compare["borderline"] = borderline_data

Dodatkowo mozna sprawdzic dzialanie "init="k-means++"" na wyniki

Stworzenie mixa mteryk i potem sprowadzenie do wartosci liniowej.

- wyprobowanie gower distance na moich danych

- zmiana danych (onehot endocer dla user oraz td zamiana na log1p(td))tak aby dalo sie wykorzystac metryki euclidean, cosinus similarity itd

### Encoding datasets

In [ ]:
ssh_df = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\test\\X_test.csv")
unique_users = ssh_df["user"].unique()
print(unique_users)
user_map = {user: idx for idx, user in enumerate(unique_users)}
list(user_map.items())[:15]

user_dummies = pd.get_dummies(original_data["user"], prefix="", prefix_sep="")

ssh_encoded = pd.concat([ user_dummies,original_data], axis=1)

reverse_user_map = {v: k for k, v in user_map.items()}
print(reverse_user_map)
ssh_encoded = ssh_encoded.rename(columns=reverse_user_map)
print(ssh_encoded)

In [ ]:
def map_user_column(series, mapping):
    if pd.api.types.is_numeric_dtype(series):
        return series.map(mapping).astype(str)
    else:
        # spróbujmy rzutować na int
        try:
            as_int = series.astype(float).astype(int)
            return as_int.map(mapping).astype(str)
        except Exception:
            # wygląda na to, że już są nazwy
            return series.astype(str)
        
original_data["user"] = map_user_column(original_data["user"], reverse_user_map)
user_dummies = pd.get_dummies(original_data["user"], prefix="", prefix_sep="")
ssh_encoded = pd.concat([ user_dummies,original_data], axis=1)
ssh_encoded = ssh_encoded.replace({True: 1, False: 0})
ssh_encoded = ssh_encoded.drop(columns=["user"])

scalar_ip_no_td = StandardScaler()
cols_to_scale = ["td", "ip_failure", "ip_success", "no_failure", "not_valid_count"]
ssh_encoded[cols_to_scale]= scalar_ip_no_td.fit_transform(ssh_encoded[cols_to_scale])
ssh_encoded.to_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\original_normalized.csv")
with open("scaler_ip_no_td.pkl", "wb") as f:
    pickle.dump(scalar_ip_no_td, f)
with open("map_user.pkl", "wb") as f:
    pickle.dump(reverse_user_map, f)

print(ssh_encoded)

In [43]:
print(smote_data.shape)
print(data_oririnal["user"].nunique())

(315, 14)
46


In [57]:
ssh_encoded = mix_data
#ssh_encoded = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\test\\X_test.csv")
#ssh_encoded = ssh_encoded.drop(columns=["Unnamed: 0"])
cols = ssh_encoded.columns.tolist()
print(cols)
colums = original_data.columns.tolist()
print(colums)
print(reverse_user_map)

['user', 'is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count', 'ip_failure', 'ip_success', 'no_failure', 'first', 'td', 'ts', 'target', 'source']
['user', 'is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count', 'ip_failure', 'ip_success', 'no_failure', 'first', 'td', 'ts', 'target', 'source']
{0: 'kamran', 1: 'student', 2: 'root', 3: 'admins', 4: 'phoenix', 5: 'piglet', 6: 'rainbow', 7: 'runner', 8: 'sam', 9: 'abc123', 10: 'passwd', 11: 'newpass', 12: 'notused', 13: 'Hockey', 14: 'internet', 15: 'asshole', 16: 'Maddock', 17: 'computer', 18: 'Mickey', 19: 'qwerty', 20: 'fiction', 21: 'orange', 22: 'tigger', 23: 'wheeling', 24: 'mustang', 25: 'admin', 26: 'jennifer', 27: 'money', 28: 'Justin', 29: 'chris', 30: 'david', 31: 'foobar', 32: 'buster', 33: 'harley', 34: 'jordan', 35: 'stupid', 36: 'apple', 37: 'fred', 38: 'summer', 39: 'sunshine', 40: 'andrew', 41: 'osamac', 42: 'gta', 43: 'adminx', 44: 'gtta', 45: 'osamax'}


In [58]:
user_dummies = pd.get_dummies(ssh_encoded["user"].astype(int), prefix="", prefix_sep="")
def normalize_col(c):
    s = str(c).strip()
    return int(s) if s.isdigit() else s
user_dummies = user_dummies.rename(columns=normalize_col)

user_dummies = user_dummies.reindex(columns=reverse_user_map, fill_value=0.0)
user_dummies = user_dummies.rename(columns=reverse_user_map)
user_dummies = user_dummies.replace({True: 1.0, False: 0.0})
print(user_dummies)
data = pd.concat([user_dummies, ssh_encoded], axis=1)
data = data.drop(columns=["user"])
cols_to_scale = ["td", "ip_failure", "ip_success", "no_failure", "not_valid_count"]
data[cols_to_scale]= scalar_ip_no_td.transform(data[cols_to_scale])
cols_to_conv = [ "is_private","is_failure","is_root", "is_valid", "first" , "target"]
for col in cols_to_conv:
    data[col] = data[col].astype("float64")
print(data.info())
print(data)
data.to_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\mix_data_normalized.csv", index=False)


      kamran  student  root  admins  phoenix  piglet  rainbow  runner  sam  \
0        0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
1        0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
2        0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
3        0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  1.0   
4        0.0      0.0   0.0     0.0      0.0     0.0      0.0     1.0  0.0   
...      ...      ...   ...     ...      ...     ...      ...     ...  ...   
1570     0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
1571     0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
1572     0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
1573     0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   
1574     0.0      0.0   0.0     0.0      0.0     0.0      0.0     0.0  0.0   

      abc123  ...  apple  fred  summer  sunshine  andrew  osama

C:\Users\mateu\AppData\Local\Temp\ipykernel_3516\2918200787.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_dummies = user_dummies.replace({True: 1.0, False: 0.0})


In [22]:
original_data = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\encoded_normalized\\original_data_normalized.csv")
expected_cols = original_data.columns.tolist()
drop_val = ["target", "source"]
for col in drop_val:
    if col in expected_cols:
        expected_cols.remove(col)
print(expected_cols)
data = data.reindex(columns=expected_cols, fill_value=0.0)
data = data.astype("float64")
print(data.info())
data.to_csv("D:\\ml\\undersampling_data\\data\\ssh\\test\\X_test_norm.csv")

['kamran', 'root', 'admins', 'phoenix', 'piglet', 'runner', 'sam', 'newpass', 'notused', 'internet', 'asshole', 'Maddock', 'computer', 'Mickey', 'qwerty', 'fiction', 'orange', 'tigger', 'wheeling', 'mustang', 'jennifer', 'money', 'chris', 'david', 'foobar', 'buster', 'harley', 'jordan', 'stupid', 'apple', 'fred', 'summer', 'andrew', 'osamac', 'gta', 'adminx', 'gtta', 'osamax', 'is_private', 'is_failure', 'is_root', 'is_valid', 'not_valid_count', 'ip_failure', 'ip_success', 'no_failure', 'first', 'td', 'ts']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 49 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   kamran           85 non-null     float64
 1   root             85 non-null     float64
 2   admins           85 non-null     float64
 3   phoenix          85 non-null     float64
 4   piglet           85 non-null     float64
 5   runner           85 non-null     float64
 6   sam        

### KMeans + centroids

In [ ]:
KM = KMeans(n_clusters=(int)(count2))

df_ = {}

for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    X_majority_reduced = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    y_majority_reduced = pd.Series([1] * (int)(count2), name="target") 
    
    df_majority = pd.concat([X_majority_reduced, y_majority_reduced], axis=1).reset_index(drop=True)
    
    
    df_majority["source"] = None
    missing_source = df_majority[df_majority["source"].isna()]
    if not missing_source.empty:
        df_majority.loc[df_majority["source"].isna(), "source"] = "centroid" 
    
    print(df_majority)
    
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True)  
    print(df_[name])
    
    #df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_centroids_data.csv", index=False)


### KMeans + NN

In [ ]:
KM = KMeans(n_clusters=(int)(count2))

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_NN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    #results_KM_SWAP_ = {}
    
    
    for i in range(len(centroids_)):
        if (len(cluster_data_[name][i])>1):
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = euclidean(centroid, row)         #tworze slwonik wartosci
                
            min_key = min(dist_, key=dist_.get)
            results_KM_NN_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
        else:
            results_KM_NN_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_NN_[name].values(), ignore_index=True)    
        
    df_y_majority = pd.Series([1] * (int)(count2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True) 
    
    df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_NN_data.csv", index=False)

### KMeans + cos

In [ ]:
KM = KMeans(n_clusters=(int)(count2))

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_COS_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    #results_KM_SWAP_ = {}
    
    
    for i in range(len(centroids_)):                #dla kazdego z centroidow
        if (len(cluster_data_[name][i])>1):       #sprawdzam czy jest wiecej niz jeden wiersz w klastrze
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = cosine_similarity(centroid, row)         #tworze slwonik wartosci
                
            min_key = min(dist_, key=dist_.get)
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
        else:
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_COS_[name].values(), ignore_index=True)    
        
    df_y_majority = pd.Series([1] * (int)(count2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True) 
    
    df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_COS_data.csv", index=False)

### K-means + cosinus + Mahalanobis distance (reduced to linear form, as it allows a simpler comparison)

In [ ]:
KM = KMeans(n_clusters=(int)(count2))

cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KM_COS_MAN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

alfa = 0.5  #wazenie

for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range ((int)(count2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    #results_KM_SWAP_ = {}
    
    
    for i in range(len(centroids_)):                #dla kazdego z centroidow
        if (len(cluster_data_[name][i])>1):       #sprawdzam czy jest wiecej niz jeden wiersz w klastrze
            cos_={}
            dist_={}
            comb_score_ = {} # results for cosine similarity and Mahalanobis distance
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                cos_[j] = cosine_similarity(centroid, row)         #tworze slwonik wartosci
                dist_[j] = distance.mahalanobis(centroid, row, np.linalg.inv(np.cov(X_train.T)))  #using Mahalanobis distance
                
                comb_score_[j] = (alfa*cos_[j] + (1-alfa)*dist_[j])
                
            min_key = min(comb_score_, key=comb_score_.get)
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
        else:
            results_KM_COS_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_COS_[name].values(), ignore_index=True)   
        
    df_y_majority = pd.Series([1] * (int)(count2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
     
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True) 
    
    #df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_COS_data.csv", index=False)

### HDBSCAN + Euclidean distance

In [ ]:
hdbscan = HDBSCAN(store_centers="centroid")

#rows_in_cluster = {}
cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
labels_={}

for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    print(X_majority.shape)
    #print(X_minority.shape)
    print(name)
    
    hdbscan_res = hdbscan.fit(X_majority)
    labels = hdbscan_res.labels_
    unique_lables = np.unique(labels)
    labels_[name] = len(unique_lables[unique_lables >=0])
    labels = len(unique_lables[unique_lables >=0])
    
    print(f"{name}: {labels_[name]}")
    centroids_ = pd.DataFrame(hdbscan_res.centroids_, columns=X_train.columns)
    #print(hdbscan_res.centroids_)
    
    centroids_hdbscan = hdbscan.fit_predict(X_majority)
    #print(centroids_hdbscan)

    #centroids
    for i in range(labels):
        rows_in_cluster = X_majority[hdbscan.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        #print(f"Cluster {i}:")
        #print(cluster_data_[name][i])
        
        
        target = count2
        #print(f"Target: {target}")
        
        per_cluster_sorted = {}
        #calculate the nieghbors for each centroid (centroid -> rows_in_cluster)
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue

        centroid = centroids_.iloc[i].to_numpy()
        # policz dystanse do centroidu
        dists = rows.apply(lambda r: euclidean(centroid, r.to_numpy()), axis=1)
        order = dists.sort_values().index.tolist()  # indeksy X_majority w kolejności rosnących dystansów
        per_cluster_sorted[i] = order

    # Round-robin wybór do target_n
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # wskaźnik który „numer” brać teraz z klastra
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  # wszystkie klastry wyczerpane
            break

    # Złóż wybrane rekordy większości (kolejność wg selekcji)
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   # <-- CHANGED
    results_[name] = maj_selected   
    #print(results_[name])
    #print("ilosc duplikaotow",results_[name].duplicated().sum())
        
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True)  
    #print(df_[name])
    
    
    #copy source from sum_all_data 
    #columns_ = list(df_[name].columns.values)
    #df_[name] = df_[name].merge(sum_all_data, on=columns_, how="left")
    print(df_[name])
    print(df_[name].duplicated().sum())

    #df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_HDBSCAN_NN_data.csv", index=False)

In [ ]:
print(labels_["smote"])

### HDBSCAN + cos

In [ ]:
hdbscan = HDBSCAN(store_centers="centroid")

#rows_in_cluster = {}
cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}


for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    print(X_majority.shape)
    #print(X_minority.shape)
    print(name)
    
    hdbscan_res = hdbscan.fit(X_majority)
    labels = hdbscan_res.labels_
    unique_lables = np.unique(labels)
    labels = len(unique_lables[unique_lables >=0])
    print(labels)
    centroids_ = pd.DataFrame(hdbscan_res.centroids_, columns=X_train.columns)
    #print(hdbscan_res.centroids_)
    
    centroids_hdbscan = hdbscan.fit_predict(X_majority)
    #print(centroids_hdbscan)

    #centroids
    for i in range(labels):
        rows_in_cluster = X_majority[hdbscan.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        #print(f"Cluster {i}:")
        #print(cluster_data_[name][i])
        
        
        target = count2
        #print(f"Target: {target}")
        
        per_cluster_sorted = {}
        #calculate the nieghbors for each centroid (centroid -> rows_in_cluster)
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue

        centroid = centroids_.iloc[i].to_numpy().reshape(1, -1)
        # policz dystanse do centroidu
        dists = pairwise_distances(rows.values, centroid, metric="cosine").ravel()
        order = rows.index[np.argsort(dists)].tolist()  # indeksy X_majority w kolejności rosnących dystansów
        per_cluster_sorted[i] = order

    # Round-robin wybór do target_n
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # wskaźnik który „numer” brać teraz z klastra
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  # wszystkie klastry wyczerpane
            break

    # Złóż wybrane rekordy większości (kolejność wg selekcji)
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   # <-- CHANGED
    results_[name] = maj_selected   
    #print(results_[name])
    #print("ilosc duplikaotow",results_[name].duplicated().sum())
        
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True)  
    #print(df_[name])
    
    
    #copy source from sum_all_data 
    #columns_ = list(df_[name].columns.values)
    #df_[name] = df_[name].merge(sum_all_data, on=columns_, how="left")
    print(df_[name])
    print(df_[name].duplicated().sum())
       
    #df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_HDBSCAN_COS_data.csv", index=False)

### KMeans (samples=HDBSCAN()) + NN

In [ ]:
KM = KMeans(n_clusters=labels_[name])

#rows_in_cluster = {}
cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KMEANS_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    KM = KMeans(n_clusters=labels_[name])
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range (labels_[name]):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    print(f"{name}")
    print(centroids_)
    #results_KM_SWAP_ = {}
    target = count2
    
    per_cluster_sorted = {}
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue
             
        centroid = centroids_.iloc[i].to_numpy()
        # policz dystanse do centroidu
        dists = rows.apply(lambda r: euclidean(centroid, r.to_numpy()), axis=1)
        order = dists.sort_values().index.tolist()  # indeksy X_majority w kolejności rosnących dystansów
        per_cluster_sorted[i] = order
        
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # wskaźnik który „numer” brać teraz z klastra
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  # wszystkie klastry wyczerpane
            break
        
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   # <-- CHANGED
    results_[name] = maj_selected     
    
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True) 
    print(df_[name]) 
    
    #df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_HDBSCAN_NN_data.csv", index=False)

### KMeans (samples=HDBSCAN()) + cosinus

In [ ]:
KM = KMeans(n_clusters=labels_[name])

#rows_in_cluster = {}
cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KMEANS_HDBSCAN_DIST_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    KM = KMeans(n_clusters=labels_[name])
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range (labels_[name]):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    print(f"{name}")
    print(centroids_)
    #results_KM_SWAP_ = {}
    target = count2
    
    per_cluster_sorted = {}
    for i in range(len(centroids_)):
        rows = cluster_data_[name][i]
        if len(rows) == 0:
            per_cluster_sorted[i] = []
            continue
        if len(rows) == 1:
            per_cluster_sorted[i] = [rows.index[0]]
            continue
             
        centroid = centroids_.iloc[i].to_numpy().reshape(1, -1)
        # policz dystanse do centroidu
        dists = pairwise_distances(rows.values, centroid, metric="cosine").ravel()
        order = rows.index[np.argsort(dists)].tolist()  # indeksy X_majority w kolejności rosnących dystansów
        per_cluster_sorted[i] = order
        
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # wskaźnik który „numer” brać teraz z klastra
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  # wszystkie klastry wyczerpane
            break
        
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   # <-- CHANGED
    results_[name] = maj_selected     
    
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True) 
    print(df_[name]) 
    
    df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_HDBSCAN_COS_data.csv", index=False)

### KMeans (samples=HDBSCAN()) + cosinus + NN

In [ ]:
KM = KMeans(n_clusters=labels_[name])

#rows_in_cluster = {}
cluster_data_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}
centroids_ = {}

results_ = {}
df_ = {}

results_KMEANS_HDBSCAN_COS_NN_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

alfa = 0.5

for name in data.keys() & compare.keys():
    X_train, y_train = data[name]
    compare_df = compare[name]
    
    KM = KMeans(n_clusters=labels_[name])
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    #X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range (labels_[name]):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        cluster_data_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    print(f"{name}")
    print(centroids_)
    #results_KM_SWAP_ = {}
    target = count2
    
    per_cluster_sorted = {}
    for i in range(len(centroids_)):                #dla kazdego z centroidow
        if (len(cluster_data_[name][i])>1):       #sprawdzam czy jest wiecej niz jeden wiersz w klastrze
            cos_={}
            dist_={}
            comb_score_ = {} # results for cosine similarity and Mahalanobis distance
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                cos_[j] = cosine_similarity(centroid, row)         #tworze slwonik wartosci
                dist_[j] = distance.mahalanobis(centroid, row, np.linalg.inv(np.cov(X_train.T)))  #using Mahalanobis distance
                
                comb_score_[j] = (alfa*cos_[j] + (1-alfa)*dist_[j])
                
            min_key = min(comb_score_, key=comb_score_.get)
            results_KMEANS_HDBSCAN_COS_NN_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
        else:
            results_KMEANS_HDBSCAN_COS_NN_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KMEANS_HDBSCAN_COS_NN_[name].values(), ignore_index=True)    
        
    selected_idx = []
    ptr = {i: 0 for i in range(len(centroids_))}                     # wskaźnik który „numer” brać teraz z klastra
    while len(selected_idx) < target:
        progressed = False
        for i in range(len(centroids_)):
            lst = per_cluster_sorted[i]
            j = ptr[i]
            if j < len(lst):
                idx = lst[j]
                if idx not in selected_idx:
                    selected_idx.append(idx)
                ptr[i] += 1
                progressed = True
                if len(selected_idx) >= target:
                    break
        if not progressed:  # wszystkie klastry wyczerpane
            break
        
    maj_selected = X_majority.loc[selected_idx].reset_index(drop=True)   # <-- CHANGED
    results_[name] = maj_selected     
    
    df_y_majority = pd.Series([1.0] * len(results_[name]), name="target")
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    columns_ = list(df_majority.columns.values)
    df_majority = df_majority.merge(sum_all_data, on=columns_, how="left")
        
    df_[name] = pd.concat([df_majority, original_data], axis=0).reset_index(drop=True) 
    print(df_[name]) 
    
    df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_HDBSCAN_COS_data.csv", index=False)

#### -------------------

In [ ]:
#KM = KMeans(n_clusters=(int)((count1+count2)/2), init="k-means++")

centroids_rows_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

centroids_ = {}

results_KM_COS_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

results_ = {}
df_ = {}

for (name, (X_train, y_train)), (_, compare_df) in zip(data.items(), compare.items()):
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    X_minority = X_train[y_train == 0]
    print(X_minority.shape)
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range ((int)((count1+count2)/2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        centroids_rows_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    #results_KM_COS_ = {}
    
    
    for i in range(len(centroids_)):                #dla kazdego z centroidow
        if (len(centroids_rows_[name][i])>1):       #sprawdzam czy jest wiecej niz jeden wiersz w klastrze
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            centroid = centroid.values.reshape(1,-1)
            for j in range(len(centroids_rows_[name][i])):
                index_ = list(centroids_rows_[name][i].index)
                row = centroids_rows_[name][i].iloc[j]
                row = row.values.reshape(1,-1)
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = cosine_similarity(centroid, row)         #tworze slwonik wartosci
                
            min_key = min(dist_, key=dist_.get)
            results_KM_COS_[name][i] = centroids_rows_[name][i].iloc[[min_key]]
            
        else:
            results_KM_COS_[name][i] = centroids_rows_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_COS_[name].values(), ignore_index=True)    
        
    df_y_majority = pd.Series([1] * (int)((count1+count2)/2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
    print(df_majority.shape)  
     
    df_X_minority = X_minority.reset_index(drop=True)
    df_y_minority = pd.Series([0] * len(X_minority), name="target")
    df_miniority = pd.concat([df_X_minority, df_y_minority], axis=1).reset_index(drop=True)
    print(df_miniority.shape)

    df_[name] = pd.concat([df_majority, df_miniority], axis=0).reset_index(drop=True)  
    print(df_[name])
    
    #copy source from sum_all_data 
    columns_ = list(df_[name].columns.values)
    df_[name] = df_[name].merge(sum_all_data, on=columns_, how="left")
    print(df_[name])  
    print(df_[name].dtypes)
    
    df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_COS_data.csv", index=False)
    
    print(f"Num duplicates: {df_[name].duplicated().sum()}")
    

In [ ]:
#KM = KMeans(n_clusters=(int)((count1+count2)/2))

centroids_rows_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

centroids_ = {}

results_KM_SWAP_ = {
    "mix": {},
    "smote": {},
    "GAN": {},
    "borderline": {}
}

results_ = {}
df_ = {}

for (name, (X_train, y_train)), (_, compare_df) in zip(data.items(), compare.items()):
     # klasteryzacja dotyczy tylko jednego ze zbiorow drugi jest przepisywany
    X_majority = X_train[y_train == 1]
    X_minority = X_train[y_train == 0]
    
    kmeans = KM.fit(X_majority)
    
    #centroids
    for i in range ((int)((count1+count2)/2)):
        rows_in_cluster = X_majority[kmeans.labels_ == i] 
        centroids_rows_[name][i] = rows_in_cluster
        
        centroids_ = pd.DataFrame(kmeans.cluster_centers_, columns=X_train.columns)
    
    #results_KM_SWAP_ = {}
    
    
    for i in range(len(centroids_)):
        if (len(centroids_rows_[name][i])>1):
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            for j in range(len(centroids_rows_[name][i])):
                index_ = list(centroids_rows_[name][i].index)
                row = centroids_rows_[name][i].iloc[j]
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = euclidean(centroid, row)         #tworze slwonik wartosci
                
            min_key = min(dist_, key=dist_.get)
            results_KM_SWAP_[name][i] = centroids_rows_[name][i].iloc[[min_key]]
            
        else:
            results_KM_SWAP_[name][i] = centroids_rows_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_KM_SWAP_[name].values(), ignore_index=True)    
        
    df_y_majority = pd.Series([1] * (int)((count1+count2)/2), name="target") 
    df_majority = pd.concat([results_[name], df_y_majority], axis=1).reset_index(drop=True)
        
     
    df_X_minority = X_minority.reset_index(drop=True)
    df_y_minority = pd.Series([0] * len(X_minority), name="target")
    df_miniority = pd.concat([df_X_minority, df_y_minority], axis=1).reset_index(drop=True)

    df_[name] = pd.concat([df_majority, df_miniority], axis=0).reset_index(drop=True)  
    
    #copy source from sum_all_data 
    columns_ = list(df_[name].columns.values)
    df_[name] = df_[name].merge(sum_all_data, on=columns_, how="left")
    print(df_[name])  
    
    df_[name].to_csv(f"D:\\ml\\undersampling_data\\data\\ssh\\reduced\\{name}_KM_NN_data.csv", index=False)

In [ ]:
    for i in range(len(centroids_)):
        
        if (len(cluster_data_[name][i])>1):
            dist_={}
            index_ = {}
            centroid = centroids_.iloc[i]
            for j in range(len(cluster_data_[name][i])):
                index_ = list(cluster_data_[name][i].index)
                row = cluster_data_[name][i].iloc[j]
                index_map = {j: idx for j, idx in enumerate(index_)}
                dist_[j] = euclidean(centroid, row)         #tworze slwonik wartosci
                
            min_key = min(dist_, key=dist_.get)
            results_HDBSCAN_DIST_[name][i] = cluster_data_[name][i].iloc[[min_key]]
            
        else:
            results_HDBSCAN_DIST_[name][i] = cluster_data_[name][i].iloc[[0]]
        
        results_[name] = pd.concat(results_HDBSCAN_DIST_[name].values(), ignore_index=True)
        print(results_[name])
        print("ilosc duplikaotow",results_[name].duplicated().sum())